In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from sklearn.model_selection import StratifiedKFold, ParameterGrid
sys.path.append(os.environ['CMS_ROOT'])
from cms_modules.utils import model_summary_to_string, args_to_dict
from cms_modules.logging import Logger

import tensorflow as tf
EarlyStopping = tf.keras.callbacks.EarlyStopping
TensorBoard = tf.keras.callbacks.TensorBoard
# ReduceLROnPlateau = tf.keras.callbacks.ReduceLROnPlateau

ecbdl14_root = '/Users/jujohnson/git/ECBDL14-Classification'
# ecbdl14_root = '/home/jjohn273/git/ECBDL14-Classification/'
sys.path.append(ecbdl14_root)
from model import create_model
from CustomCallbacks import KerasRocAucCallback

### Define Model Config

In [ ]:
config = {
    'hidden_layers': [128, 128, 64, 32],
    'learn_rate': 0.001,
    'batch_size': 256,
    'dropout_rate': 0.2,
    'batchnorm': True
}

epochs = 40
debug = True

### Define I/O Paths

In [ ]:
# inputs
data_path = os.path.join(ecbdl14_root, 'data/ecbdl14.onehot.sample.hdf')
data_key = 'train'
# outputs
now = datetime.datetime.today()
ts = now.strftime("%m%d%y-%H%M%S")
architecture_output = 'model-architecture.json'

### Init Logger

In [ ]:
log_file = f'logs/{ts}.txt'
logger = Logger(log_file)
logger.log_time('Starting trainer')
logger.write_to_file()

### Load Data

In [ ]:
df = pd.read_hdf(data_path, data_key)
logger.log_time(f'Loaded data with shape {df.shape}').write_to_file()
if debug:
    y, x = df[:10000]['target'], df[:10000].drop(columns=['target'])
else:
    y, x = df['target'], df.drop(columns=['target'])

In [ ]:
def write_model(model, path):
    json = model.to_json()
    with open(path, 'w') as out:
        out.write(json)

### Iterate Over Runs

In [ ]:
for run in runs:
    logger.log_time(f'Starting run {run}')).write_to_file()
    # prepare input data
    x_train, y_train = x.iloc[train_index].values, y.iloc[train_index].values
    x_valid, y_valid = x.iloc[validate_index].values, y.iloc[validate_index].values
    input_dim = x_train.shape[1]
        
    # create model and save architecture to file if does not exist
    dnn = create_model(input_dim, config)
    if not os.path.isfile(architecture_output):
        write_model(dnn, architecture_output)
        print(f'Model architecture saved to {architecture_output}')

    # train model
    logger.log_time('Starting training...').write_to_file()
    dnn.fit(x_train, y_train, epochs=epochs, verbose=0)
    logger.log_time('Trainin complete!').write_to_file()
    
    # save weights
    out = f'{experiment}-{run}-model.h5'
    dnn.save_weights(out)
    logger.log_time(f'Model weights saved to {out}')

In [ ]:
logger.log_time('Job complete...').write_to_file()